In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/Final datasets/USAID/combined_usaid_country.csv')
target = pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/FINAL ML/SUPERVISED LEARNING/target_encoding.csv')

In [3]:
df.dropna(inplace=True)

In [4]:
target

,Unnamed: 0,Country Name,Result_Inv
0,0,Afghanistan,No Positive Change
1,1,Albania,Positive Change
2,2,Angola,No Positive Change
3,3,Armenia,Positive Change
4,4,Azerbaijan,No Positive Change
...,...,...,...
66,66,Ukraine,Positive Change
67,67,Vietnam,No Positive Change
68,68,West Bank and Gaza,Positive Change
69,69,Yemen,No Positive Change


In [5]:
df

,country,duration_mean,topic0_mean,topic1_mean,topic2_mean,topic3_mean,topic4_mean,dominant_topic_4_count,dominant_topic_3_count,dominant_topic_2_count,...,awd_amount_median_low,awd_amount_median,awd_amount_median_high,awd_amount_high,awd_amount_very_high,corruption_initial,stability_initial,numberofprojects,change_stability,change_corruption
0,Afghanistan,3.071429,0.036429,0.037857,0.066071,0.176429,0.679643,23,4,1,...,3,1,7,7,5,High Corruption,Unstable,28,Worse,Worse
1,Albania,3.750000,0.037500,0.078750,0.053750,0.308750,0.523750,5,3,0,...,6,2,0,0,0,High Corruption,Unstable,8,Same,Same
2,Angola,5.000000,0.030000,0.030000,0.030000,0.030000,0.870000,1,0,0,...,0,1,0,0,0,High Corruption,Unstable,1,Same,Same
3,Armenia,4.250000,0.035000,0.035000,0.035000,0.035000,0.862500,4,0,0,...,1,1,0,0,0,High Corruption,Unstable,4,Same,Same
4,Azerbaijan,2.800000,0.032000,0.032000,0.032000,0.150000,0.752000,4,1,0,...,1,1,0,0,0,High Corruption,Unstable,5,Same,Same
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Ukraine,3.375000,0.035000,0.041250,0.038750,0.541250,0.338750,2,6,0,...,3,2,0,0,0,High Corruption,Unstable,8,Same,Same
70,Vietnam,3.500000,0.040000,0.040000,0.040000,0.845000,0.040000,0,2,0,...,0,2,0,0,0,High Corruption,Unstable,2,Same,Same
71,West Bank and Gaza,4.200000,0.034000,0.034000,0.047000,0.313000,0.580000,7,3,0,...,1,4,2,2,0,High Corruption,Very Unstable,10,Same,Same
72,Yemen,4.000000,0.036667,0.036667,0.096667,0.036667,0.800000,3,0,0,...,0,1,1,0,0,High Corruption,Very Unstable,3,Same,Same


In [6]:
result = df.merge(target, left_on = 'country',right_on = 'Country Name', how = 'right')

In [7]:
result.dropna(inplace=True)

In [8]:
result.drop(['country','Unnamed: 0'],axis=1,inplace=True)

In [9]:
col = result.columns.to_list()

In [10]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67 entries, 0 to 66
Data columns (total 62 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   duration_mean                                                            67 non-null     float64
 1   topic0_mean                                                              67 non-null     float64
 2   topic1_mean                                                              67 non-null     float64
 3   topic2_mean                                                              67 non-null     float64
 4   topic3_mean                                                              67 non-null     float64
 5   topic4_mean                                                              67 non-null     float64
 6   dominant_topic_4_count                                                   67 

In [11]:
list(result.select_dtypes(include=['object']).columns)
## TIHNK ABOUT COLUMNS TO INCLUDE AS LABEL ENCODER AND ONE HOT ENCODER

['max_count_topic_categories',
 'max_count_sector_categories',
 'corruption_initial',
 'stability_initial',
 'change_stability',
 'change_corruption',
 'Country Name',
 'Result_Inv']

In [12]:
## DATA PREP

col = ['corruption_initial',
 'stability_initial',
 'change_stability',
 'change_corruption',]


cor_ord = ['High Corruption', 'Very High Corruption']
sta_ord = ['Unstable', 'Very Unstable']
chg_ord = ['Worse', 'Same', 'Better']


from sklearn.preprocessing import LabelEncoder
y = result['Result_Inv']
X = result.drop(['Result_Inv'],axis=1)

def ordered_labels(df, col, order):
    df[col] = df[col].astype('category')
    df[col] = df[col].cat.reorder_categories(order, ordered=True)
    df[col] = df[col].cat.codes.astype(int)
    
    
    
ordered_labels(X, 'corruption_initial', cor_ord)
ordered_labels(X, 'stability_initial', sta_ord)
ordered_labels(X, 'change_stability', chg_ord)
ordered_labels(X, 'change_corruption', chg_ord)


dummies = ['max_count_topic_categories',
 'max_count_sector_categories',
    'Country Name']

for i in dummies:
    X = pd.concat([X, pd.get_dummies(X[i])],axis=1).drop(i,axis=1)

In [13]:
X

,duration_mean,topic0_mean,topic1_mean,topic2_mean,topic3_mean,topic4_mean,dominant_topic_4_count,dominant_topic_3_count,dominant_topic_2_count,dominant_topic_1_count,...,Sierra Leone,Sri Lanka,Thailand,Turkmenistan,Uganda,Ukraine,Vietnam,West Bank and Gaza,Yemen,Zambia
0,3.071429,0.036429,0.037857,0.066071,0.176429,0.679643,23.0,4.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,3.750000,0.037500,0.078750,0.053750,0.308750,0.523750,5.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,5.000000,0.030000,0.030000,0.030000,0.030000,0.870000,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4.250000,0.035000,0.035000,0.035000,0.035000,0.862500,4.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,2.800000,0.032000,0.032000,0.032000,0.150000,0.752000,4.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,3.375000,0.035000,0.041250,0.038750,0.541250,0.338750,2.0,6.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
63,3.500000,0.040000,0.040000,0.040000,0.845000,0.040000,0.0,2.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
64,4.200000,0.034000,0.034000,0.047000,0.313000,0.580000,7.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
65,4.000000,0.036667,0.036667,0.096667,0.036667,0.800000,3.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
#RIDGE

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)


from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
lm = LogisticRegression(fit_intercept = False,penalty='l2', solver='liblinear')
lm.fit(scaled_X, y)

from sklearn.metrics import confusion_matrix,classification_report
confusionMatrix = confusion_matrix (y, lm.predict(scaled_X))
print(confusionMatrix)
print('-------------------------------\n')
classification_report = classification_report (y, lm.predict(scaled_X))
print(classification_report)

[[32  0]
 [ 0 35]]
-------------------------------

                    precision    recall  f1-score   support

No Positive Change       1.00      1.00      1.00        32
   Positive Change       1.00      1.00      1.00        35

          accuracy                           1.00        67
         macro avg       1.00      1.00      1.00        67
      weighted avg       1.00      1.00      1.00        67



In [22]:
df = pd.DataFrame(list(zip(X.columns.to_list(), lm.coef_.reshape(-1).tolist())), columns =['Variable', 'Coefficient']) 

In [23]:
df_significant = df['Coefficient'].reset_index(drop=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    print(df_significant)

                                              Variable  Coefficient
0                                        duration_mean     0.000000
1                                          topic0_mean     0.000000
2                                          topic1_mean     0.000000
3                                          topic2_mean     0.646245
4                                          topic3_mean     0.000000
5                                          topic4_mean     0.000000
6                               dominant_topic_4_count     0.000000
7                               dominant_topic_3_count     0.000000
8                               dominant_topic_2_count     0.000000
9                               dominant_topic_1_count    -0.099132
10                              dominant_topic_0_count     0.000000
11                          Independent_Agencies_count     0.000000
12                                   Rule_of_Law_count     0.000000
13             Elections_and_Political_Processes

In [17]:
## Using Lasso we don't get the count of projects as a significant variable. 

In [18]:
#RIDGE

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)


from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
lm = LogisticRegression(fit_intercept = False,penalty='l1', solver='liblinear')
lm.fit(scaled_X, y)

from sklearn.metrics import confusion_matrix,classification_report
confusionMatrix = confusion_matrix (y, lm.predict(scaled_X))
print(confusionMatrix)
print('-------------------------------\n')
classification_report = classification_report (y, lm.predict(scaled_X))
print(classification_report)

[[32  0]
 [ 0 35]]
-------------------------------

                    precision    recall  f1-score   support

No Positive Change       1.00      1.00      1.00        32
   Positive Change       1.00      1.00      1.00        35

          accuracy                           1.00        67
         macro avg       1.00      1.00      1.00        67
      weighted avg       1.00      1.00      1.00        67



In [19]:
df = pd.DataFrame(list(zip(X.columns.to_list(), lm.coef_.reshape(-1).tolist())), columns =['Variable', 'Coefficient']) 

In [20]:
df_significant = df['Coefficient'].reset_index(drop=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    print(df_significant)

                                             Variable  Coefficient
0                                         topic2_mean     0.646245
1                                         Media_count     0.174690
2                 intervention_type_Enforcement_count     0.624039
3                                   awd_amount_median     1.305672
4                                  corruption_initial     0.829622
5                                    change_stability     0.197133
6                              dominant_topic_2_count     0.094611
7                                 Civil_Society_count     0.229350
8   Local_Government_countMedia_countGovernance_count     0.024283
9              Rule_of_Law_countEconomic_Growth_count     0.102241
10  Rule_of_Law_countLocal_Government_countCivil_S...     0.059329
11  Rule_of_Law_countLocal_Government_countGoverna...     0.330946
12                                            Armenia     0.004137
13                                           Bulgaria     0.34